# Policy Gradients & Policy Optimisation


Up to now, we have been focused in **value-based RL**. This means, we learn the value function, and use the policy that is implicitly obtained from it. What we will do now is to turn it around, and go for **policy-based RL**: we just learn directly the policy, without the value function. A intermediate class of methods is called **Actor-Critic**, where both policy and value function are learnt. This later approach will not be discussed here.

Why would this make sense? Well, sometimes we may just want to know what to do (left/right), not how bad or good it is to do either.

So far, we have obtained policies through the $Q$-function (e.g. using $\epsilon$-greedy), but we can parameterise directly the **policy**:
	$$ \pi_\theta(s,a) = \mathbb P(a \ | \ s, \theta) $$
As with the value functions, if we use a differentiable parametrisation of the policy, we can update it by gradient descent.


For example, we can assume the policy has the following structure:
$$\pi_\theta(s,a) = \frac{\mathrm{exp}(h(s,a,\theta))}{\sum_{a'}\mathrm{exp}(h(s,a',\theta))}$$

where 

$$h(s,a,\theta) = \theta^T\cdot\mathbf{x}(s,a)$$.



The advantages of policy gradient are:
* **Stronger convergence guarantees** (at least to a local optimum). 
* Effective in continuous action spaces.
* No maximization of the $Q$ function is needed, which also means that the update is less "aggressive" than in $Q$-learning.



### Objective function
- **Goal**: find the best parameter $\theta$ for $\pi_\theta(s,a)$, according a performance functional $J(\theta)$.
- Update according to the rule:
$$ \theta_{t+1} = \theta_t + \nabla \hat{J}(\theta_t)$$
where $\hat{J}$ is a stochastic approximation of the gradient (obtained, for example, by gradient **ascent**.


## Derivative Free Methods

Deep Learning's success relies on backpropagation and gradients. Gradients, however, are usually bad news for reinforcement learning for a number of reasons: two issues are that rewards come far in the future, so it is not obvious how to calculate derivatives, and the other issue is the existence of local optima.

Luckily, there is a large class of optimization methods that do not use of the gradients (the expensive part of the computation), and are easy to parallelize. We will discuss two methods which have shown promising results.




### Cross-Entropy Method

* Blackbox optimization for $f(w)$.
* Start with $\mu, \sigma$, `batch_size`, `elite_frac`.
* Do forever:
  * Sample `batch_size` possible values for $w$.
  * Evaluate $f$ on the sampled values.
  * Keep the top `elite_frac` of those. These are `elite_set`.
  * $\mu \leftarrow$ `np.mean(elite_set)`
  * $\sigma \leftarrow$ `np.std(elite_set)`




### Natural Evolution Strategies

It might not be good idea to take some elite solutions to resample and throw away the rest, since "weak" parameters are valuable because they tell us what *not* to do. An alternative approach from CEM is to maximise the expected value of the reward, so that when we would sample from that population, we would very likely get lucky and pick a good parameter.    

[It has been shown](https://blog.openai.com/evolution-strategies/) that NES is comparable results to fancier algorithms in a number of tasks



### NES Algorithm

* Start with $\sigma$, $\alpha$, `n_estimators`.
* Sample $\epsilon_1, \epsilon_2$ from a normal distribution with mean $0$ and variance $1$. These are **vectors**.
* Calculate return $f(w+\sigma \cdot \epsilon_i)$.
*  $w \leftarrow w + \alpha\cdot \frac{1}{N}\sum_{i=1}^n\left(\frac{f(w+\sigma\epsilon_i)-f(w)}{\sigma}\epsilon_i\right)$ 
* Stopping criteria: function changes below a threshold, or, in the case of OpenAI Gym, performance benchmark is reached.

In [ ]:
## Code sample: NES for FrozenLake
import numpy as np
import gym
np.random.seed(0)

env= gym.make('CartPole-v0')

# the function we want to optimize
def f(w):
    n_iter = 200
    total_reward = 0
    for it in range(n_iter):
        state = env.reset()
        done = False
        while not done:
            action = 1 if np.matmul(state,w) < 0 else 0
            new_state, reward, done, _ = env.step(action)
            total_reward+=reward
            state = new_state
    return total_reward/(1+it)

# hyperparameters
npop = 100 # population size
sigma = 0.1 # noise standard deviation
alpha = 0.001 # learning rate

w = 2*np.random.random(4)-1 # our initial guess is random

i = 0

while f(w) <50:
    i+=1
  # print current fitness of the most likely parameter setting
    if i % 5 == 0:
        print('iter %d. w: %s, reward: %f' %(i, str(w),  f(w)))

  # initialize memory for a population of w's, and their rewards
    N = np.random.randn(npop, 4) # samples from a normal distribution N(0,1)
    R = np.zeros(npop)
    for j in range(npop):
        w_try = w + sigma*N[j] # jitter jw using gaussian of sigma 0.1
        R[j] = f(w_try) # evaluate the jittered version
    
    # standardize the rewards to have a gaussian distribution
    A = (R - np.mean(R)) / np.std(R)
    # perform the parameter update. The matrix multiply below
    # is just an efficient way to sum up all the rows of the noise matrix N,
    # where each row N[j] is weighted by A[j]
    w = w + alpha/(npop*sigma+1) * np.dot(N.T, A)
  
env.close()  

In [ ]:
## Code sample: Cross entropy method for CartPole

import numpy as np
import gym

# Task settings:
env = gym.make('CartPole-v0') # Change as needed
num_steps = 500 # maximum length of episode
# Alg settings:
n_iter = 100 # number of iterations of CEM
batch_size = 25 # number of samples per batch
elite_frac = 0.2 # fraction of samples used as elite set


# Initialize mean and standard deviation
dim_theta = (env.observation_space.shape[0]+1) * env.action_space.n
theta_mean = np.zeros(dim_theta)
theta_std = np.ones(dim_theta)


def make_policy(theta):
    n_states = env.observation_space.shape[0]
    n_actions = env.action_space.n
    W = theta[0 : n_states * n_actions].reshape(n_states, n_actions)
    b = theta[n_states * n_actions : None].reshape(1, n_actions)
    
    def policy_fn(state):
        y = state.dot(W) + b
        action = y.argmax()
        return action
        
    return policy_fn

def run_episode(theta, num_steps, render=False):
    total_reward = 0
    state = env.reset()
    policy = make_policy(theta)
    for t in range(num_steps):
        a = policy(state)
        state, reward, done, _ = env.step(a)
        total_reward += reward
        if render and t%3==0: env.render()
        if done: break
    return total_reward

def noisy_evaluation(theta):
    total_reward = run_episode(theta, num_steps)
    return total_reward


# Now, for the algorithms
for it in range(n_iter):
    # Sample parameter vectors
    thetas = np.random.normal(theta_mean, theta_std, (batch_size,dim_theta))
    rewards = [noisy_evaluation(theta) for theta in thetas]
    # Get elite parameters
    n_elite = int(batch_size * elite_frac)
    elite_inds = np.argsort(rewards)[batch_size - n_elite:batch_size]
    elite_thetas = [thetas[i] for i in elite_inds]
    # Update theta_mean, theta_std
    theta_mean = np.mean(elite_thetas,axis=0)
    theta_std = np.std(elite_thetas,axis=0)
    #print("Mean reward f: {}. \
    # Max reward: {}".format(np.mean(rewards), np.max(rewards)))
    run_episode(theta_mean, num_steps, render=True)

## CEM in steroids: DeepCEM

Instead of generating the samples from a normal distribution, we can use an arbitrary (and unknown) distribution approximated by neural networks! Each step of the optimization generates a sample produced by a neural network and adjusts its weights based on the outcome. Example from [Yandex Data School](https://github.com/yandexdataschool/Practical_RL)



In [ ]:
import seaborn as sns
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import gym
from tqdm import tqdm
import numpy as np

class DeepCEM():
    def __init__(self, initial_state, \
                 n_actions, \
                 clf=MLPClassifier(hidden_layer_sizes=(20,20), \
                                   activation='tanh', \
                                  warm_start=True, #keep progress between .fit(...) calls
                                  max_iter=1 #make only 1 iteration on each .fit(...)
                             )):
        self.clf = clf
        
        #initialize clf
        clf.fit([initial_state]*n_actions,range(n_actions))
        
    def policy(self, state):
        probs = self.clf.predict_proba([state])[0]
        action = np.random.choice(n_actions, p=probs)
        return action

    def train(self,n_iter):
        n_episodes = 100 
        percentile = 75
        
        for i in tqdm(range(n_iter)):
            #generate new episodes
            episodes = [play_episode(self) \
                        for _ in range(n_episodes)]

            batch_states, batch_actions, batch_rewards = \
            map(np.array,zip(*episodes))

            reward_threshold = np.percentile(batch_rewards,70)
            idxs = [i for \
                    i in range(len(batch_rewards)) \
                    if batch_rewards[i]>reward_threshold]

            elite_states, elite_actions = \
            np.concatenate(batch_states[idxs],axis=0), \
            np.concatenate(batch_actions[idxs],axis=0)
            self.clf.fit(elite_states, elite_actions)

            if np.mean(batch_rewards)> 195:
                print("Solved.")


def play_episode(agent, max_iter=1000,render=False):
    states,actions = [],[]
    total_reward = 0
    state = env.reset()
    for _ in range(max_iter):
        # choose the action according to the policy
        action = agent.policy(state)
        new_state,reward,done,info = env.step(action)
        if render:
            env.render()
        # record sessions
        states.append(state)
        actions.append(action)
        total_reward+=reward
        state = new_state
        if done: 
            break
    return states,actions,total_reward

env = gym.make("CartPole-v0").env  
env.reset()
n_actions = env.action_space.n

agent = DeepCEM(initial_state=env.reset(), n_actions=2)
agent.train(n_iter=100)
states, actions, rewards= play_episode(agent, render=True)

